In [6]:
# 1. 문제정의
# 평가: roc-auc
# 최종파일: result.csv(컬럼 1개 pred, 1확률값)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

train = pd.read_csv("hr_train.csv")
test = pd.read_csv("hr_test.csv")

# train.info()
train.shape, test.shape

# 결측치
train.isnull().sum()

# 결측치 처리 (object형 - 'X'로 채움)
train = train.fillna('X')
test = test.fillna('X')

train.isnull().sum()

# 레이블 인코딩 
from sklearn.preprocessing import LabelEncoder
# 레이블 객체 생성 
encoder = LabelEncoder()

cols = train.select_dtypes(include="object").columns
cols

for col in cols:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.fit_transform(test[col])
  
# 데이터 선택 
target = train.pop("target")
train.info()

# 학습용(훈련/검증) 데이터 분할
from sklearn.model_selection import train_test_split
# X = train.drop("target", axis=1)
# y = train["target"]

# train_test_split?
X_train, X_test, y_train, y_test = train_test_split(
    train, target, test_size=0.3, random_state=11)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 모델 생성 및 학습, 예측 
from sklearn.ensemble import RandomForestClassifier
# RandomForestClassifier?
model = RandomForestClassifier(random_state=11)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(pred)

# 모델의 성능 평가
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test, pred)
print("score =", score)

# 평가용(테스트용)
pred2 = model.predict(test)
print(pred2)

df = pd.DataFrame({
    'pred': pred2
})

# csv 파일 저장 
df.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city                    15326 non-null  int32  
 2   city_development_index  15326 non-null  float64
 3   gender                  15326 non-null  int32  
 4   relevent_experience     15326 non-null  int32  
 5   enrolled_university     15326 non-null  int32  
 6   education_level         15326 non-null  int32  
 7   major_discipline        15326 non-null  int32  
 8   experience              15326 non-null  int32  
 9   company_size            15326 non-null  int32  
 10  company_type            15326 non-null  int32  
 11  last_new_job            15326 non-null  int32  
 12  training_hours          15326 non-null  int64  
dtypes: float64(1), int32(10), int64(2)
memory usage: 958.0 KB
(10728, 13) (4598, 13) (10728,) (

,pred
0,0.0
1,0.0
2,1.0
3,0.0
4,0.0
...,...
3827,0.0
3828,0.0
3829,0.0
3830,0.0


In [5]:
# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
# 훈련, 테스트 데이터 합치기
combined = pd.concat([train, test])
cols = train.select_dtypes(include='object').columns

for col in cols:
    le = LabelEncoder()
    combined[col] = le.fit_transform(combined[col])

n_train = len(train)
train = combined[:n_train]
test = combined[n_train:]

# id 제거 (성능 떨어짐)
train = train.drop('enrollee_id', axis=1)
test = test.drop('enrollee_id', axis=1)
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 15326 entries, 0 to 15325
Columns: 194 entries, city_development_index to last_new_job_never
dtypes: bool(192), float64(1), int64(1)
memory usage: 3.2 MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 3832 entries, 0 to 3831
Columns: 194 entries, city_development_index to last_new_job_never
dtypes: bool(192), float64(1), int64(1)
memory usage: 808.3 KB
None
